<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-and-preamble" data-toc-modified-id="Import-and-preamble-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import and preamble</a></span></li><li><span><a href="#Apply-region-mapping" data-toc-modified-id="Apply-region-mapping-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Apply region mapping</a></span><ul class="toc-item"><li><span><a href="#To-do-list" data-toc-modified-id="To-do-list-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>To do list</a></span></li><li><span><a href="#Identify-regions-with-free-text" data-toc-modified-id="Identify-regions-with-free-text-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Identify regions with free text</a></span></li><li><span><a href="#Identify-countries-with-no-region-mapping" data-toc-modified-id="Identify-countries-with-no-region-mapping-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Identify countries with no region mapping</a></span></li><li><span><a href="#Export-to-CSV" data-toc-modified-id="Export-to-CSV-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Export to CSV</a></span></li></ul></li><li><span><a href="#Number-completed-by-region-(PAPs-only,-nonPAPs-only,-or-none)" data-toc-modified-id="Number-completed-by-region-(PAPs-only,-nonPAPs-only,-or-none)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Number completed by region (PAPs only, nonPAPs only, or none)</a></span></li><li><span><a href="#Datatable-mockup" data-toc-modified-id="Datatable-mockup-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Datatable mockup</a></span></li><li><span><a href="#View-last-added-rows-of-df.-Compare-to-Tableau" data-toc-modified-id="View-last-added-rows-of-df.-Compare-to-Tableau-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>View last added rows of df. Compare to Tableau</a></span></li></ul></div>

<b>Instructions for Use</b>

1. In the GAWS folder, create a new folder with the current date, it will remain empty.
2. Move all of "english, french, spanish".csv to the last dated empty folder as backup.
3. Download labeled CSV files from redcap for each of the languages, with labels. Name them english.csv, french.csv, spanish.csv.

# Import and preamble

In [326]:
import requests
import pandas as pd
import numpy as np
from io import StringIO

In [327]:
data = {
    'token': '98F364325C69DC1309C77B0ACF890221',
    'content': 'record',
    'format': 'csv',
    'type': 'flat',
    'csvDelimiter': '',
    'rawOrLabel': 'label',
    'rawOrLabelHeaders': 'label',
    'exportCheckboxLabel': 'false',
    'exportSurveyFields': 'true',
    'exportDataAccessGroups': 'false',
    'returnFormat': 'json'
}
r = requests.post('https://redcap.ucsf.edu/api/',data=data)

eng = pd.read_csv(StringIO(r.text))

data = {
    'token': 'C5974B05A8318646A0D3B193192C99F4',
    'content': 'record',
    'format': 'csv',
    'type': 'flat',
    'csvDelimiter': '',
    'rawOrLabel': 'label',
    'rawOrLabelHeaders': 'label',
    'exportCheckboxLabel': 'false',
    'exportSurveyFields': 'true',
    'exportDataAccessGroups': 'false',
    'returnFormat': 'json'
}
r = requests.post('https://redcap.ucsf.edu/api/',data=data)

fr = pd.read_csv(StringIO(r.text))

data = {
    'token': '2169B4519B2EE75E42FEE8BAF6E16A92',
    'content': 'record',
    'format': 'csv',
    'type': 'flat',
    'csvDelimiter': '',
    'rawOrLabel': 'label',
    'rawOrLabelHeaders': 'label',
    'exportCheckboxLabel': 'false',
    'exportSurveyFields': 'true',
    'exportDataAccessGroups': 'false',
    'returnFormat': 'json'
}
r = requests.post('https://redcap.ucsf.edu/api/',data=data)

sp = pd.read_csv(StringIO(r.text))

regions = pd.read_csv('regions.csv')

Insert 4 columns in the foreign language files, so the number of columns match

In [328]:
def insertcol(dataf):
    dataf.insert(13, 'dummycol1',np.nan)
    dataf.insert(13, 'dummycol2',np.nan)
    dataf.insert(13, 'dummycol3',np.nan)
    dataf.insert(13, 'dummycol4',np.nan)

insertcol(fr)
insertcol(sp)

This is the english language column headers

In [329]:
collist = eng.columns.to_list()

Set the foreign language column headers to English, so they can be appended together

In [330]:
fr.columns=collist
sp.columns=collist
#fr = fr.rename(columns=[collist])


Append all the dataframes

Also rename the country column to Country

In [331]:
df = pd.DataFrame()
df = eng.copy() 
df = df.append(sp)
df = df.append(fr)
df = df.rename(columns={"1.1 Select the country* about which you are completing this survey: *List from&nbsp;World Health Organization&nbsp;country list. If you do not see your country, please select 'other' at the end of the list.": "Country"})

# Apply region mapping

## To do list

* Once Macedonia comes in, the name will be wrong bc of redcap using 'former republic of yugoslavia'. Needs to be changed programatically to North Macedonia

## Identify regions with free text

Which countries are not selected in the drop down? These won't be mapped to region, as region is based on the column 'Country'

In [332]:
df[['Country','1.1.2 What is the name of your country?']][pd.notnull(df['1.1.2 What is the name of your country?'])]

,Country,1.1.2 What is the name of your country?
2,Your Country (Other),Serbia
51,Your Country (Other),"Taiwan, Republic of China"
127,Your Country (Other),Singapore


If "1.1.2 What is the name of your country?" is notnull, copy the value to "Country", so region can be applied

In [333]:
df['Country'] = df.apply(lambda x: x['1.1.2 What is the name of your country?'] if pd.notnull(x['1.1.2 What is the name of your country?']) else x['Country'], axis=1)


Now we merge with the "Region" dataframe, on Country, to map the countries to region

In [334]:
df = df.merge(regions, how='left', on="Country", indicator=True)

Check again to see if countries with free text are now copied to 'Country' column, and if any countries are not mapped to Region

In [335]:
df[['Country','1.1.2 What is the name of your country?','Region']][pd.notnull(df['1.1.2 What is the name of your country?'])]

,Country,1.1.2 What is the name of your country?,Region
2,Serbia,Serbia,European Region
51,"Taiwan, Republic of China","Taiwan, Republic of China",Western Pacific Region
127,Singapore,Singapore,Western Pacific Region


## Identify countries with no region mapping

In [336]:
df[['Country','Region']][pd.isnull(df['Region'])]

,Country,Region
50,NaN,NaN
66,NaN,NaN
111,NaN,NaN
117,NaN,NaN
120,NaN,NaN
122,NaN,NaN
128,NaN,NaN
132,NaN,NaN
139,NaN,NaN
146,NaN,NaN


## Export to CSV

In [337]:
df.to_csv('current gaws.csv', index=False)

In [338]:
df.iloc[:,3]
col_date = 2
col_country = 3
col_contributor = 6
col_papnum = 17 # num of PAPs
col_pap_nonspec = 29 #num of nonspecialist npaps
col_nursenum = 43 #num of nurse providers
col_nurse2num = 58 # num of nurse providers (2nd type)
col_nurse3num = 73 # num of nurse providers (3rd type)
col_nurse4num = 88 # num of nurse providers (4th type)
col_npapnum = 100 # num of NPAP providers
col_region = 122

Export subset table to csv for display on webpage

In [339]:
t1 = df.iloc[:,[col_date,col_region,col_country,col_contributor,col_papnum , col_pap_nonspec , col_nursenum , col_nurse2num , col_nurse3num, col_nurse4num , col_npapnum ]]
tempcollist = ['Date completed', 'Region','Country','Contributor','# PAPs','# Nonspecialty PAPs','# Nurse1','# Nurse 2','#Nurse 3', '# Nurse 4','# NPAPs']
t1.columns = tempcollist
t1.to_csv('maintable.csv', index=False)

# Number completed by region (PAPs only, nonPAPs only, or none)

First find if each record has data for PAP, nurses, or NPAPs

In [340]:
newvarlist = ['haspap','haspap_nonspec','hasnurse1', 'hasnurse2','hasnurse3','hasnurse4', 'hasnpap']
columnlist = [col_papnum,col_pap_nonspec, col_nursenum, col_nurse2num, col_nurse3num, col_nurse4num, col_npapnum]

for i,j in zip(newvarlist,columnlist):
    df[i] = np.where(df.iloc[:,j] >0, 1, 0)

Next, get list of countries with PAPs, with nurses, or with NPAPs, and make them lists

In [341]:
haspap_true = df['Country'][(df['haspap'] == 1)|(df['haspap_nonspec'])].to_list()
hasnpap_true = df['Country'][(df['hasnurse1'] == 1)|(df['hasnurse2'] == 1)|(df['hasnurse3']==1)|(df['hasnurse4']==1)|(df['hasnpap']==1)].to_list()

Function that takes the 'regions' file with all countries and checks if that country is in the NPAP list, PAP list, or both, or neither

In [342]:
# 1 = has pap only
# 2 = has nurse/npap only
# 3 = has both
# 0 = has neither

def providertype(var):
    if (var in haspap_true) & (var in hasnpap_true):
        return 3
    elif (var in haspap_true) & (var not in hasnpap_true):
        return 1
    elif (var not in haspap_true) & (var in hasnpap_true):
        return 2
    else:
        return 0

regions['provider type'] = regions['Country'].apply(providertype)

In [343]:
# sample display of regions mapped to Western Pacific, with provider type
regions[regions['Region']=='South-East Asia Region']

,Region,Country,Country_iso,provider type
14,South-East Asia Region,Bangladesh,BGD,1
21,South-East Asia Region,Bhutan,BTN,3
80,South-East Asia Region,India,IND,1
81,South-East Asia Region,Indonesia,IDN,3
110,South-East Asia Region,Maldives,MDV,1
122,South-East Asia Region,Myanmar,MMR,0
125,South-East Asia Region,Nepal,NPL,1
166,South-East Asia Region,Sri Lanka,LKA,1
176,South-East Asia Region,Thailand,THA,3
177,South-East Asia Region,Timor-Leste,TLS,3


In [344]:
regions.to_csv('haspap.csv', index = False)

# Datatable mockup

# View last added rows of df. Compare to Tableau

In [345]:
eng.tail()

,Record ID,Survey Identifier,Survey Timestamp,"1.1 Select the country* about which you are completing this survey: *List from&nbsp;World Health Organization&nbsp;country list. If you do not see your country, please select 'other' at the end of the list.",1.1.2 What is the name of your country?,"1.2 Are are you interested in being a collaborator in the&nbsp;Global Anesthesia Workforce Study Group (GAWS)? (To be co-author on GAWS publications, your participation must meet requirements outlined by ICMJE. )",1.3 Please provide your name,1.4 Please provide your email address,1.5 What is your role in the (physician or non-physician) national anesthesia society (or association) of [country] ?,2.1.1 What types of anesthesia providers are currently working in [country]? Select all that apply(Please use provider terminology as defined at the top of this survey) (choice=Certified Specialist Physician Anesthesia Providers (or trainees)),...,How many intensive care medicine specialists are there in [country]? (i.e. total number of physicians who have completed a certified intensive care medicine training program) (Please include both those who have trained in [country] or in another country) (Do not include trainees. Please include both members and non-members of the national society),How many specialist physician anesthesia providers are there in [country] who have completed a certified intensive care medicine training program? (Please include both those who have trained in [country] or in another country),"In your opinion, is the intensive care medicine training provided during specialist physician anesthesia training (i.e. residency) in [country] sufficient for a graduate to function autonomously as an intensive care medicine provider?",Is there a certified training pathway for becoming an intensive care nurse in [country]?,Which of the following training pathways for becoming an intensive care nurse exist in [country]: (select all true statement) (choice=Dedicated intensive care training program after completion of nursing training),Which of the following training pathways for becoming an intensive care nurse exist in [country]: (select all true statement) (choice=Intensive care training during nursing training),Which of the following training pathways for becoming an intensive care nurse exist in [country]: (select all true statement) (choice=Intensive care training during nurse anesthesia provider training),Which of the following training pathways for becoming an intensive care nurse exist in [country]: (select all true statement) (choice=Other),"Please let us know if there is additional information you want to share about the anesthesia or critical care workforce in [country], including any additional anesthesia provider cadres that you have not yet told us about.",Complete?
141,190,NaN,2021-08-25 17:39:13,Samoa,NaN,No,Fa'amuamua Arasi,fbarasi@protonmail.com,Member and part of leadership,Checked,...,0.0,4.0,No,No,Unchecked,Unchecked,Unchecked,Unchecked,Thank you,Complete
142,191,NaN,[not completed],Croatia,NaN,Yes,Josip Brusic,josip.brusic2@ri.t-com.hr,Member and part of leadership,Checked,...,NaN,NaN,NaN,NaN,Unchecked,Unchecked,Unchecked,Unchecked,NaN,Incomplete
143,192,NaN,2021-08-25 14:26:27,Nauru,NaN,No,Pavlo Zelenin,pavelzelenin1965@gmail.com,Member but not part of leadership,Checked,...,0.0,1.0,No,No,Unchecked,Unchecked,Unchecked,Unchecked,NaN,Complete
144,193,NaN,2021-08-26 03:49:39,Croatia,NaN,Yes,Josip Brusic,josip.brusic2@ri.t-com.hr,Member and part of leadership,Checked,...,412.0,412.0,Yes,No,Unchecked,Unchecked,Unchecked,Unchecked,yes,Complete
145,194,NaN,2021-08-26 11:46:54,Croatia,NaN,Yes,Josip Brusic,josip.brusic2@ri.t-com.hr,Member and part of leadership,Checked,...,290.0,0.0,Yes,No,Unchecked,Unchecked,Unchecked,Unchecked,Yes,Complete


In [346]:
fr.tail()

,Record ID,Survey Identifier,Survey Timestamp,"1.1 Select the country* about which you are completing this survey: *List from&nbsp;World Health Organization&nbsp;country list. If you do not see your country, please select 'other' at the end of the list.",1.1.2 What is the name of your country?,"1.2 Are are you interested in being a collaborator in the&nbsp;Global Anesthesia Workforce Study Group (GAWS)? (To be co-author on GAWS publications, your participation must meet requirements outlined by ICMJE. )",1.3 Please provide your name,1.4 Please provide your email address,1.5 What is your role in the (physician or non-physician) national anesthesia society (or association) of [country] ?,2.1.1 What types of anesthesia providers are currently working in [country]? Select all that apply(Please use provider terminology as defined at the top of this survey) (choice=Certified Specialist Physician Anesthesia Providers (or trainees)),...,How many intensive care medicine specialists are there in [country]? (i.e. total number of physicians who have completed a certified intensive care medicine training program) (Please include both those who have trained in [country] or in another country) (Do not include trainees. Please include both members and non-members of the national society),How many specialist physician anesthesia providers are there in [country] who have completed a certified intensive care medicine training program? (Please include both those who have trained in [country] or in another country),"In your opinion, is the intensive care medicine training provided during specialist physician anesthesia training (i.e. residency) in [country] sufficient for a graduate to function autonomously as an intensive care medicine provider?",Is there a certified training pathway for becoming an intensive care nurse in [country]?,Which of the following training pathways for becoming an intensive care nurse exist in [country]: (select all true statement) (choice=Dedicated intensive care training program after completion of nursing training),Which of the following training pathways for becoming an intensive care nurse exist in [country]: (select all true statement) (choice=Intensive care training during nursing training),Which of the following training pathways for becoming an intensive care nurse exist in [country]: (select all true statement) (choice=Intensive care training during nurse anesthesia provider training),Which of the following training pathways for becoming an intensive care nurse exist in [country]: (select all true statement) (choice=Other),"Please let us know if there is additional information you want to share about the anesthesia or critical care workforce in [country], including any additional anesthesia provider cadres that you have not yet told us about.",Complete?
10,12,NaN,2021-06-10 12:47:02,Burkina Faso,NaN,Yes,KI Kélan Bertille,bertilleki@yahoo.fr,Membre et faisant partie des dirigeants,Checked,...,57.0,57.0,No,Yes,Unchecked,Unchecked,Unchecked,Checked,NaN,Complete
11,13,NaN,[not completed],Benin,NaN,No,OROU Jérémie,orjetim@yahoo.fr,Autre,Unchecked,...,NaN,NaN,NaN,NaN,Unchecked,Unchecked,Unchecked,Unchecked,NaN,Incomplete
12,14,NaN,[not completed],Benin,NaN,Yes,OROU Jérémie,orjetim@yahoo.fr,Membre sans faire partie des dirigeants,Checked,...,NaN,NaN,NaN,NaN,Unchecked,Unchecked,Unchecked,Unchecked,NaN,Incomplete
13,15,NaN,2021-07-21 01:55:54,Benin,NaN,Yes,OROU Jérémie,orjetim@yahoo.fr,Membre sans faire partie des dirigeants,Unchecked,...,NaN,NaN,NaN,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Les deux dernières rubriques n'existent pas au...,Complete
14,16,NaN,2021-08-26 03:31:40,Morocco,NaN,Yes,mouhajir mohamed,mouhajir10@gmail.com,Membre et faisant partie des dirigeants,Checked,...,NaN,NaN,NaN,Yes,Unchecked,Checked,Checked,Unchecked,effectif insuffisant et absence des référentie...,Complete


In [347]:
sp.tail()

,Record ID,Survey Identifier,Survey Timestamp,"1.1 Select the country* about which you are completing this survey: *List from&nbsp;World Health Organization&nbsp;country list. If you do not see your country, please select 'other' at the end of the list.",1.1.2 What is the name of your country?,"1.2 Are are you interested in being a collaborator in the&nbsp;Global Anesthesia Workforce Study Group (GAWS)? (To be co-author on GAWS publications, your participation must meet requirements outlined by ICMJE. )",1.3 Please provide your name,1.4 Please provide your email address,1.5 What is your role in the (physician or non-physician) national anesthesia society (or association) of [country] ?,2.1.1 What types of anesthesia providers are currently working in [country]? Select all that apply(Please use provider terminology as defined at the top of this survey) (choice=Certified Specialist Physician Anesthesia Providers (or trainees)),...,How many intensive care medicine specialists are there in [country]? (i.e. total number of physicians who have completed a certified intensive care medicine training program) (Please include both those who have trained in [country] or in another country) (Do not include trainees. Please include both members and non-members of the national society),How many specialist physician anesthesia providers are there in [country] who have completed a certified intensive care medicine training program? (Please include both those who have trained in [country] or in another country),"In your opinion, is the intensive care medicine training provided during specialist physician anesthesia training (i.e. residency) in [country] sufficient for a graduate to function autonomously as an intensive care medicine provider?",Is there a certified training pathway for becoming an intensive care nurse in [country]?,Which of the following training pathways for becoming an intensive care nurse exist in [country]: (select all true statement) (choice=Dedicated intensive care training program after completion of nursing training),Which of the following training pathways for becoming an intensive care nurse exist in [country]: (select all true statement) (choice=Intensive care training during nursing training),Which of the following training pathways for becoming an intensive care nurse exist in [country]: (select all true statement) (choice=Intensive care training during nurse anesthesia provider training),Which of the following training pathways for becoming an intensive care nurse exist in [country]: (select all true statement) (choice=Other),"Please let us know if there is additional information you want to share about the anesthesia or critical care workforce in [country], including any additional anesthesia provider cadres that you have not yet told us about.",Complete?
11,16,NaN,2021-05-22 12:16:46,Cuba,NaN,Yes,Dr. Antonio de Jesús Cabrera Prats,cabrerap@infomed.sld.cu,miembro y parte del liderazgo,Checked,...,NaN,1200.0,No,Yes,Checked,Checked,Checked,Unchecked,La especialidad de cuidados intensivos es inde...,Complete
12,17,NaN,2021-05-22 12:18:42,Cuba,NaN,Yes,Dr. Antonio de Jesús Cabrera Prats,cabrerap@infomed.sld.cu,miembro y parte del liderazgo,Unchecked,...,NaN,1200.0,No,Yes,Unchecked,Unchecked,Unchecked,Unchecked,La especialidad de cuidados intensivos es inde...,Complete
13,18,NaN,[not completed],Spain,NaN,Yes,Emilia Guasch,emiguasch@hotmail.com,miembro pero no parte del liderazgo,Checked,...,NaN,NaN,NaN,NaN,Unchecked,Unchecked,Unchecked,Unchecked,NaN,Incomplete
14,19,NaN,2021-07-27 16:50:56,Spain,NaN,Yes,Antonia Ballesteros Barrado,toballesteros@gmail.com,miembro y parte del liderazgo,Checked,...,2214.0,2214.0,Yes,No,Unchecked,Unchecked,Unchecked,Unchecked,Al no existir reconocimiento de la especialida...,Complete
15,20,NaN,[not completed],Spain,NaN,NaN,NaN,NaN,NaN,Unchecked,...,NaN,NaN,NaN,NaN,Unchecked,Unchecked,Unchecked,Unchecked,NaN,Incomplete


# Figures

## Progress bar

In [360]:
import plotly.express as px
haspap = pd.read_csv('haspap.csv')

In [362]:
def recode(x):
    if x == 0:
        return 'None'
    elif x == 1:
        return 'PAP only'
    elif x == 2:
        return 'NPAP only'
    elif x == 3:
        return 'both'

haspap['provider2'] = haspap['provider type'].apply(recode)

In [363]:
cxtab_normal = pd.crosstab(haspap['Region'], haspap['provider2'], normalize='index') 
cxtab_count = pd.crosstab(haspap['Region'], haspap['provider2']) 

In [364]:
t4_normal = cxtab_normal.reset_index().melt(id_vars='Region',value_name='percent')
t4_count = cxtab_count.reset_index().melt(id_vars='Region',value_name='count')
t4_normal['count'] = t4_count['count']

In [365]:
fig = px.bar(t4_normal, y='Region', x='percent', color='provider2', 
             orientation='h',
            color_discrete_map={"None": px.colors.qualitative.Dark2[1], "NPAP only": "cornflowerblue", "PAP only": px.colors.qualitative.Pastel[4], "both":px.colors.qualitative.Pastel[9]},
             #color_discrete_sequence=px.colors.qualitative.Pastel2,
             hover_data=['count'],
            labels={"provider2":"Provider type"},
            category_orders={"provider2":['PAP only','both','NPAP only','None']},
            title='Completion by Region',
             text='count'
            )


fig.show()

## Map

In [418]:
fig = px.choropleth(locations=haspap['Country'], locationmode='country names', scope='world',color=haspap['provider type'])

fig.show()

In [370]:
haspap

,Region,Country,Country_iso,provider type,provider2
0,Eastern Mediterranean Region,Afghanistan,AFG,0,None
1,European Region,Albania,ALB,0,None
2,African Region,Algeria,DZA,0,None
3,European Region,Andorra,AND,0,None
4,African Region,Angola,AGO,0,None
...,...,...,...,...,...
195,Western Pacific Region,Vietnam,VNM,0,None
196,African Region,West Africa,WLF,0,None
197,Eastern Mediterranean Region,Yemen,YEM,1,PAP only
198,African Region,Zambia,ZMB,3,both


In [430]:
fig = px.choropleth(locations=haspap['Country_iso'], locationmode='ISO-3', scope='world', color=haspap['provider2'], color_discrete_map={'None':'red','both':'blue','NPAP only':'yellow','PAP only':'green'})
fig.show()

TypeError: choropleth() got an unexpected keyword argument 'color_discrete_map'

In [385]:
regions

,Region,Country,Country_iso,provider type
0,Eastern Mediterranean Region,Afghanistan,AFG,0
1,European Region,Albania,ALB,0
2,African Region,Algeria,DZA,0
3,European Region,Andorra,AND,0
4,African Region,Angola,AGO,0
...,...,...,...,...
195,Western Pacific Region,Vietnam,VNM,0
196,African Region,West Africa,WLF,0
197,Eastern Mediterranean Region,Yemen,YEM,1
198,African Region,Zambia,ZMB,3
